In [40]:
import pandas as pd
import random

In [24]:
player_matches = pd.read_csv("../Data/20100101_20191129_player_matches.csv")

In [85]:
def get_game_for_row(row):
    if row['home_game']:
        return row['opponent_key'] + "@" + row['team_key']
    else:
        return row['team_key'] + "@" + row['opponent_key']

In [105]:
def get_player_salary(min_fp, max_fp, player_fp):
    MAX_SALARY = 12000
    MIN_SALARY = 3500

    return round(MIN_SALARY + MAX_SALARY * (player_fp - min_fp) / max_fp + (random.random() - 0.5) * 1000, -2)

In [106]:
def generate_salaries(daily_pms):
    max_fp = daily_pms['fp_l5'].max()
    min_fp = daily_pms['fp_l5'].min()
    
    player_fps = list(daily_pms['fp_l5'])
    salaries = []
    
    for player_fp in player_fps:
        salaries.append(get_player_salary(min_fp, max_fp, player_fp))
    
    return salaries

In [107]:
def generate_positions(num):
    pos = []

    for i in range(num):
        rand = random.random()
        if rand < 0.20:
            pos.append("C")
        elif rand < 0.45:
            pos.append("PG")
        elif rand < 0.70:
            pos.append("SF")
        else:
            pos.append("SG")

    return pos

In [143]:
def create_fanduel_competition_for_date(player_matches, date):
    daily_pms = player_matches[player_matches.date == date]
    
    if len(daily_pms) == 0:
        return None
    
    fantasy_df = pd.DataFrame()
    fantasy_df['player_id'] = daily_pms['player_id']
    fantasy_df['Position'] = generate_positions(len(daily_pms))
    fantasy_df['Nickname'] = daily_pms['name']
    fantasy_df['Salary'] = generate_salaries(daily_pms)
    fantasy_df['Game'] = daily_pms.apply(get_game_for_row, axis = 1)
    fantasy_df['Team'] = daily_pms['team_key']
    fantasy_df['Opponent'] = daily_pms['opponent_key']
    fantasy_df['FP'] = daily_pms['fp']
    
    fantasy_df.to_csv("../testing/fake_fanduel_competitions/" + "competition_" + date + ".csv", index = False)
    
    return fantasy_df

In [144]:
player_matches[player_matches.date == "2013-10-30"]

,Unnamed: 0,3pa,3pa_l5,3pm,3pm_l5,ast,ast_l5,blk,blk_l5,date,...,season_start_year,secs,secs_l5,stl,stl_l5,team_key,tvr,tvr_l5,win,year
63573,63573,2,7.0,1,3.0,9,8.4,1,0.600000,2013-10-30,...,2013,1304,2624.000000,0,0.6,BKN,4,3.200000,False,2013
63574,63574,0,0.2,0,0.2,1,1.4,4,3.000000,2013-10-30,...,2013,2017,2350.200000,0,1.0,BKN,1,2.200000,False,2013
63575,63575,0,0.2,0,0.0,3,3.4,1,1.000000,2013-10-30,...,2013,1577,2105.800000,2,1.0,BKN,2,3.200000,False,2013
63576,63576,2,6.6,0,1.6,9,7.2,0,0.200000,2013-10-30,...,2013,1855,1940.000000,2,0.8,CLE,2,3.400000,True,2013
63577,63577,4,6.8,3,2.0,2,5.0,0,0.600000,2013-10-30,...,2013,1791,2593.200000,0,0.8,BKN,1,5.200000,False,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63764,63764,0,0.0,0,0.0,0,1.0,0,0.666667,2013-10-30,...,2013,1312,1040.666667,2,1.0,UTA,1,0.666667,False,2013
63765,63765,3,1.6,0,0.6,6,1.4,0,0.400000,2013-10-30,...,2013,1890,1051.000000,0,0.8,UTA,3,0.400000,False,2013
63766,63766,5,1.8,2,0.8,0,0.6,2,0.200000,2013-10-30,...,2013,1176,838.400000,0,0.6,UTA,2,0.400000,False,2013
63767,63767,5,2.2,2,0.6,1,1.6,0,0.200000,2013-10-30,...,2013,1885,1376.400000,2,1.0,OKC,1,0.800000,True,2013


In [145]:
dates = [
    "2012-03-01",
    "2013-02-07",
    "2013-10-30",
    "2015-10-08",
    "2012-10-08",
    "2013-11-17",
    "2014-11-17",
]

In [146]:
for date in dates:
    create_fanduel_competition_for_date(player_matches, date)